In [10]:
import pandas as pd
import numpy as np
import datetime
import json
import pivottablejs
from IPython.display import HTML
import IPython.core.display as di
from IPython.display import display
from IPython.display import Markdown as md
from IPython.display import display, HTML

In [2]:
#!pip install --user pyarrow

In [3]:
df = pd.read_csv('metadata/noronet_metadata.csv', low_memory=False, sep=',')#, index_col=0)

In [21]:
#display(HTML(df.head().to_html()))

In [25]:
# Example for multiple information stored in a column
list(df['Genus / Genogroup'].tail())

['Caliciviridae Norovirus GII, Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GII, Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GII, Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GII, Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GII, Caliciviridae Norovirus GII']

We should group sequences based on their Genogroups, and there are 4 columns for this info

In [4]:
s = set()
for i in range(1,5):
    s = s.union(set(df['seq %d: Genus / Genogroup'%i].unique()))
s

{'Caliciviridae Norovirus GI',
 'Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GIII',
 'Caliciviridae Norovirus GIV',
 'Caliciviridae Norovirus GV',
 'Caliciviridae Norovirus GVII',
 'Caliciviridae Sapovirus GI',
 'Caliciviridae Sapovirus GII',
 'Unassigned',
 nan}

Unknown entries and Sapovirus are not useful so we remove them

In [12]:
available_genogroups = list(s)
available_genogroups.remove(np.nan)
available_genogroups.remove('Unassigned')
available_genogroups.remove('Caliciviridae Sapovirus GI')
available_genogroups.remove('Caliciviridae Sapovirus GII')
available_genogroups

['Caliciviridae Norovirus GI',
 'Caliciviridae Norovirus GIV',
 'Caliciviridae Norovirus GVII',
 'Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GV',
 'Caliciviridae Norovirus GIII']

In [15]:
# Nan values to 'Unassigned'
for i in range(1,5):
    genogroup = 'seq %d: Genus / Genogroup'%i
    df.loc[df[genogroup].isna(), genogroup] = 'Unassigned'

#### & Assumption: the following columns store multiple informations, which is alredy stored in another column anyway

In [29]:
#First get rid of the columns that contain a summary of other columns
drop_cols = ['ORF1', 'ORF1 variant', 'ORF2', 'ORF2 variant', 'Genus / Genogroup' ]
df = df.drop(drop_cols, axis=1)

#### & Assumption: each seq (1,2,3,4) entry is a different virus
We will create a new DataFrame, but each seqX will be in a separate row.

For that we need to determine, which columns/metadata will be copied along

In [30]:
copycols = list(df.columns)
#print(copycols)
copycols = ['User', 'Institute', 'Database ID', 'Country', 'Submission Date','Last Update', 'Outbreak Nr', 'Sample Date', 'Dutch lab Y/N', 'Nosocomial infection Y/N (sampledate > 2days after date of hospitalization)', 'Outbreak or sporadic case O/S',
 'Source of the sample', 'Specify source', 'Specify animal', 'Suspected country of infection', 'Date of first disease', 'Geo-coded location', 'Suspected mode of transmission',
 'Specify other mode of transmission', 'Food item', 'Specify food item', 'Setting of the outbreak', 'Specify setting', 'Point source transmission Y/N', 'Date of point source transmission',
 'Nr of persons affected', 'Nr of persons at risk', 'Nr of cases deceased', 'Nr of cases hospitalized due to infection', 'Nr of cases of ages 0 to 4', 'Nr of cases of ages 5 to 14',
 'Nr of cases of ages 15 to 64', 'Nr of cases of age 65 or older', 'Nr of cases with vomiting', 'Nr of cases with diarrhea', 'Nr of cases with vomiting AND diarrhea',
 'Mixed infection Y/N', 'Specify other pathogen(s)', 'Nr of samples tested', 'Nr of PCR positive samples', 'Nr of PCR negative samples', 'Included in II.4 P2 capsid surveillance',
 'fasta_id', 'reference_id', 'fragment_begin', 'Genus-Genogroup', 'ORF1', 'ORF1_variant', 'ORF2', 'ORF2_variant', 'Reference_sequence_for_positions', 'Cluster']

In [31]:
tdfs = []
for si in range(1,5):
    rename_dict = {'seq %d: fasta id'%(si): 'fasta_id',
                   'seq %d: reference id'%(si): 'reference_id',
                   'seq %d: fragment begin'%(si): 'fragment_begin',
                   'seq %d: fragment end'%(si): 'fragment_end',
                   'seq %d: Genus / Genogroup'%(si): 'Genus-Genogroup',
                   'seq %d: ORF1'%(si): 'ORF1',
                   'seq %d: ORF1 variant'%(si): 'ORF1_variant',
                   'seq %d: ORF2'%(si): 'ORF2',
                   'seq %d: ORF2 variant'%(si): 'ORF2_variant',
                   'seq %d: Reference sequence for positions'%(si): 'Reference_sequence_for_positions',
                   'seq %d: Cluster'%(si): 'Cluster'}
    tdfs.append(df.rename(rename_dict, axis=1)[copycols])
#     for ir, r in tdf.iterrows():
#         gengroup = 'Genus-Genogroup'
#         if r[gengroup] in available_genogroups:
#             ndf.append(r, ignore_index=True)

ndf = pd.concat(tdfs)

In [32]:
print(ndf.shape)
ndf = ndf[ndf['Genus-Genogroup'].notna()]
ndf = ndf.reset_index(level=0, drop=True)
print(ndf.shape)
print(ndf[ndf['Genus-Genogroup']=='Unassigned'].shape)

(82912, 52)
(82912, 52)
(56586, 52)


## Data cleaning
* drop columns with irrelevant data

In [33]:
# ?

* Get rid of columns with no value at all

In [34]:
for c in ndf.columns:
    if ndf[c].notna().sum()==0:
        print("Dropping column: %s"%c)
        ndf.drop(c, axis=1, inplace=True)
    

Dropping column: Nosocomial infection Y/N (sampledate > 2days after date of hospitalization)


In [35]:
# Check relevancy
drop_irrelevant_cols = ['Dutch lab Y/N',  'Nr of cases with vomiting AND diarrhea']
ndf.drop(drop_irrelevant_cols, axis=1, inplace=True)

In [37]:
# Looks better this way
colname = 'Suspected country of infection'
ndf.loc[ndf[colname]=='Same as reporting country', colname] = ndf.loc[ndf[colname]=='Same as reporting country', 'Country']

#### split ORFs into genogroup and genotype

In [38]:
ndf['ORF1'].unique()

array([nan, 'GII.P7', 'GII.P4', 'GII.Pa', 'GII.P3', 'GII.P1', 'GII.P12',
       'GII.P2', 'GI.Pb', 'Could not assign', 'GI.P2', 'GII.P5', 'GI.P4',
       'GI.P3', 'GII.Pg', 'GII.P6', 'GII.Pf', 'GII.P21', 'GII.Ph',
       'GI.P5', 'GII.P16', 'GII.Pr', 'GII.P8', 'GII.Pj', 'GII.Pe',
       'GI.P7', 'GII.P13', 'GI.Pd', 'GI.Pa', 'GI.P1', 'GI.Pf', 'GII.P22',
       'GI.P9', 'GII.P17', 'GI.P6', 'GII.Pm', 'GI.P8', 'GII.P20',
       'GII.Pc', 'GII.Pq', 'GII.P11', 'GII.P30 (GII.Pc)',
       'GI.P13 (GI.Pd)', 'GII.P31 (GII.Pe)', 'GI.P14 (GI.Pf)',
       'GII.P21 (GII.Pb)', 'GI.P11 (GI.Pb)', 'GII.P15'], dtype=object)

#### & Assumption: For these below we use the one in the bracket
'GII.P30 (GII.Pc)'
'GI.P13 (GI.Pd)'
'GII.P31 (GII.Pe)'
'GI.P14 (GI.Pf)'
'GII.P21 (GII.Pb)'
'GI.P11 (GI.Pb)'

In [39]:
ndf.loc[ndf['ORF1']=='Could not assign', 'ORF1'] = 'Unknown'
ndf.loc[ndf['ORF2']=='Could not assign', 'ORF2'] = 'Unknown'

In [40]:
ndf['ORF1_genotype'] = ndf['ORF1'].apply(lambda x: x.split(".")[-1] if type(x) != np.float else 'Unknown')
ndf['ORF1_genogroup'] = ndf['ORF1'].apply(lambda x: x.split(".")[0] if type(x) != np.float else 'Unknown')
ndf['ORF2_genotype'] = ndf['ORF2'].apply(lambda x: x.split(".")[-1] if type(x) != np.float else 'Unknown')
ndf['ORF2_genogroup'] = ndf['ORF2'].apply(lambda x: x.split(".")[0] if type(x) != np.float else 'Unknown')

In [41]:
# Correct Typos
ndf.loc[ndf['ORF1_genotype']=='Pb)', 'ORF1_genotype'] = 'Pb'
ndf.loc[ndf['ORF1_genotype']=='Pc)', 'ORF1_genotype'] = 'Pc'
ndf.loc[ndf['ORF1_genotype']=='Pd)', 'ORF1_genotype'] = 'Pd'
ndf.loc[ndf['ORF1_genotype']=='Pe)', 'ORF1_genotype'] = 'Pe'
ndf.loc[ndf['ORF1_genotype']=='Pf)', 'ORF1_genotype'] = 'Pf'

### Sort out columns depending on their type of data
* with boolean (e.g. yes/no)
* with only strings
* with few integers
* with a range of numbers
* with datetime data

Then we could put columns with fewer unique values in the columns of the pivottable

In [42]:
booleans = ['Outbreak or sporadic case O/S', 'Point source transmission Y/N', 'Mixed infection Y/N', 'Included in II.4 P2 capsid surveillance']
numbers = [ 'Nr of persons affected', 'Nr of persons at risk', 'Nr of cases deceased', 'Nr of cases hospitalized due to infection', 'Nr of cases of ages 0 to 4',
           'Nr of cases of ages 5 to 14', 'Nr of cases of ages 15 to 64', 'Nr of cases of age 65 or older', 'Nr of cases with vomiting', 'Nr of cases with diarrhea',
            'Nr of samples tested', 'Nr of PCR positive samples', 'Nr of PCR negative samples']
geo = [ 'Institute', 'Country', 'Suspected country of infection']
dates = ['Submission Date', 'Last Update', 'Sample Date','Date of first disease','Date of point source transmission',]
ids = ['User', 'Database ID', 'Outbreak Nr']
pathological = ['Source of the sample', 'Specify source', 'Suspected mode of transmission', 'Specify other mode of transmission', 'Food item', 'Specify food item',
 'Setting of the outbreak', 'Specify setting', 'Specify other pathogen(s)']
seq_data =['fasta_id', 'reference_id', 'fragment_begin', 'Genus-Genogroup', 'ORF1_genogroup', 'ORF1_genotype', 'ORF1_variant', 'ORF2_genogroup', 'ORF2_genotype', 'ORF2_variant', 'Reference_sequence_for_positions', 'Cluster']
etc = ['Geo-coded location']

### Convert some columns into categorycal type

In [44]:
for c in ['Country', 'ORF1_genogroup', 'ORF2_genogroup', 'ORF1_genotype', 'ORF2_genotype', 'Genus-Genogroup', 'Setting of the outbreak']:
    ndf[c] = ndf[c].astype('category')

### convert date to datetime

In [45]:
dates

['Submission Date',
 'Last Update',
 'Sample Date',
 'Date of first disease',
 'Date of point source transmission']

In [47]:
# a = pd.to_datetime(ndf['Sample Date'], errors='coerce')

for i in range(len(dates)):
    print(i)
    ndf.loc[:, dates[i]] = pd.to_datetime(ndf[dates[i]], errors='coerce')

0
1
2
3
4


In [48]:
ndf[dates].head()

,Submission Date,Last Update,Sample Date,Date of first disease,Date of point source transmission
0,1992-01-01,NaT,1992-01-01,1992-01-01,NaT
1,1994-01-01,NaT,1994-01-01,1994-01-01,NaT
2,1994-01-01,NaT,1994-01-01,1994-01-01,NaT
3,1997-01-01,NaT,1997-01-01,1997-01-01,NaT
4,1997-01-01,NaT,1997-01-01,1997-01-01,NaT


In [49]:
ndf = ndf[ndf['Sample Date'].notna()]
# reindex entries
ndf = ndf.reset_index(drop=True)

### Saving data

In [52]:
# dataframe
#output = "noronet_clean.fr"
output = "noronet_all_clean.fr"
#output_categories = "categories.json"
output_categories = "categories_all.json"
ndf.to_feather(output)

# categories
dict_categories = {'geo':geo,
                  'booleans':booleans,
                  'numbers': numbers,
                  'dates': dates,
                   'ids': ids,
                   'pathological': pathological,
                   'seq_data': seq_data,
                   'etc': etc
                  }

with open(output_categories, 'w') as f:
    f.write(json.dumps(dict_categories))


### Lets see each categories in detail, and check if we didn't miss any detail

In [53]:
#booleans
for c in booleans:
    print(ndf[c].unique())
ndf[booleans].info()

['O' 'S' nan]
[nan 'No' 'Yes']
[nan 'No' 'Yes']
[nan 'Yes' 'No']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82900 entries, 0 to 82899
Data columns (total 4 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Outbreak or sporadic case O/S            56752 non-null  object
 1   Point source transmission Y/N            31332 non-null  object
 2   Mixed infection Y/N                      39852 non-null  object
 3   Included in II.4 P2 capsid surveillance  12032 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [54]:
#numbers
dict_numbers_unique_values = { c: len(ndf[c].unique()) for c in numbers}
print("Number of unique values for each column")
for k in dict_numbers_unique_values.keys():
    print(k, "\t",dict_numbers_unique_values[k])

print("\nInformations about the numbers category")
ndf[numbers].info()

Number of unique values for each column
Nr of persons affected 	 276
Nr of persons at risk 	 548
Nr of cases deceased 	 30
Nr of cases hospitalized due to infection 	 74
Nr of cases of ages 0 to 4 	 52
Nr of cases of ages 5 to 14 	 97
Nr of cases of ages 15 to 64 	 105
Nr of cases of age 65 or older 	 128
Nr of cases with vomiting 	 96
Nr of cases with diarrhea 	 103
Nr of samples tested 	 79
Nr of PCR positive samples 	 59
Nr of PCR negative samples 	 56

Informations about the numbers category
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82900 entries, 0 to 82899
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Nr of persons affected                     21236 non-null  object
 1   Nr of persons at risk                      11620 non-null  object
 2   Nr of cases deceased                       9044 non-null   object
 3   Nr of cases hospitalized due t

In [55]:
#geo
dict_unique_values = { c: len(ndf[c].unique()) for c in geo}
print("Number of unique values for each column")
for k in dict_unique_values.keys():
    print(k, "\t",dict_unique_values[k])

print("\nInformations about the geo data")
ndf[geo].info()

Number of unique values for each column
Institute 	 43
Country 	 24
Suspected country of infection 	 29

Informations about the geo data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82900 entries, 0 to 82899
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Institute                       82900 non-null  object  
 1   Country                         82900 non-null  category
 2   Suspected country of infection  75908 non-null  object  
dtypes: category(1), object(2)
memory usage: 1.3+ MB


In [56]:
#dates
dict_unique_values = { c: len(ndf[c].unique()) for c in dates}
print("Number of unique values for each column")
for k in dict_unique_values.keys():
    print(k, "\t",dict_unique_values[k])

print("\nInformations about the numbers category")
ndf[dates].info()

Number of unique values for each column
Submission Date 	 1852
Last Update 	 7189
Sample Date 	 4480
Date of first disease 	 3456
Date of point source transmission 	 247

Informations about the numbers category
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82900 entries, 0 to 82899
Data columns (total 5 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Submission Date                    82900 non-null  datetime64[ns]
 1   Last Update                        28776 non-null  datetime64[ns]
 2   Sample Date                        82900 non-null  datetime64[ns]
 3   Date of first disease              42320 non-null  datetime64[ns]
 4   Date of point source transmission  1104 non-null   datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 3.2 MB


In [57]:
#pathological
dict_unique_values = { c: len(ndf[c].unique()) for c in pathological}
print("Number of unique values for each column")
for k in dict_unique_values.keys():
    print(k, "\t",dict_unique_values[k])

print("\nInformations about the pathological category")
ndf[pathological].info()

Number of unique values for each column
Source of the sample 	 6
Specify source 	 28
Suspected mode of transmission 	 7
Specify other mode of transmission 	 82
Food item 	 10
Specify food item 	 185
Setting of the outbreak 	 12
Specify setting 	 695
Specify other pathogen(s) 	 111

Informations about the pathological category
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82900 entries, 0 to 82899
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Source of the sample                79640 non-null  object  
 1   Specify source                      516 non-null    object  
 2   Suspected mode of transmission      36608 non-null  object  
 3   Specify other mode of transmission  524 non-null    object  
 4   Food item                           2780 non-null   object  
 5   Specify food item                   1576 non-null   object  
 6   Setting of the outbreak        

In [58]:
#seq
print("\nInformations about the the sequences")
ndf[seq_data].info()


Informations about the the sequences
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82900 entries, 0 to 82899
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   fasta_id                          26523 non-null  object  
 1   reference_id                      25977 non-null  object  
 2   fragment_begin                    25977 non-null  float64 
 3   Genus-Genogroup                   82900 non-null  category
 4   ORF1_genogroup                    82900 non-null  category
 5   ORF1_genotype                     82900 non-null  category
 6   ORF1_variant                      7704 non-null   object  
 7   ORF2_genogroup                    82900 non-null  category
 8   ORF2_genotype                     82900 non-null  category
 9   ORF2_variant                      6221 non-null   object  
 10  Reference_sequence_for_positions  25977 non-null  object  
 11  Cluster         